In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time

# Set up Selenium WebDriver
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode (no UI)
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")

service = Service("chromedriver.exe")  # Update path if necessary
driver = webdriver.Chrome(service=service, options=chrome_options)

# Player details
player_name = "virat-kohli"
player_id = "253802"
url = f"https://www.espncricinfo.com/cricketers/{player_name}-{player_id}/bowling-batting-stats"

# Function to get player stats
def get_player_stats(url):
    driver.get(url)
    time.sleep(5)  # Wait for page to load

    # Get player name
    try:
        player_full_name = driver.find_element(By.TAG_NAME, "h1").text
    except:
        player_full_name = player_name

    # Locate batting & bowling tables
    tables = driver.find_elements(By.CLASS_NAME, "ds-w-full.ds-table.ds-table-xs.ds-table-fixed")

    if len(tables) < 2:
        print(f"❌ Stats table not found for {player_full_name}")
        return None

    batting_table, bowling_table = tables[0], tables[1]

    # Initialize stats dictionary
    stats = {"Player": player_full_name}

    formats = ["Test", "ODI", "T20I", "overall"]

    # Extract batting stats
    for row in batting_table.find_elements(By.TAG_NAME, "tr")[1:]:
        cols = row.find_elements(By.TAG_NAME, "td")
        format_name = cols[0].text.strip().lower()

        if format_name in [fmt.lower() for fmt in formats]:
            stats[f"Bat_Avg_{format_name}"] = cols[5].text.strip() if cols[5].text.strip() != "-" else "0"
            stats[f"Bat_SR_{format_name}"] = cols[7].text.strip() if cols[7].text.strip() != "-" else "0"

    # Extract bowling stats
    for row in bowling_table.find_elements(By.TAG_NAME, "tr")[1:]:
        cols = row.find_elements(By.TAG_NAME, "td")
        format_name = cols[0].text.strip().lower()

        if format_name in [fmt.lower() for fmt in formats]:
            stats[f"Bowl_Avg_{format_name}"] = cols[5].text.strip() if cols[5].text.strip() != "-" else "0"
            stats[f"Bowl_SR_{format_name}"] = cols[7].text.strip() if cols[7].text.strip() != "-" else "0"

    return stats

# Scrape data for the player
player_data = get_player_stats(url)

# Convert to DataFrame and save to CSV
if player_data:
    df = pd.DataFrame([player_data])
    df.to_csv(f"{player_name}_stats.csv", index=False)
    print(f"✅ Data saved to {player_name}_stats.csv")

# Close WebDriver
driver.quit()


NoSuchDriverException: Message: Unable to obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location
